## Load PyEpiDAG

In [1]:
import epidag as dag

## Define a DAG

Compose a script

```
PCore Exp1 {
    # Definitions of nodes
}
```

Then, .. parse to a DAG

In [2]:
script = '''
PCore Exp1 {
    n
    a = 0.5 
    b ~ beta(1, 1)
    c = min([a, b])
    y ~ binom(n, c)
}
'''

js = dag.bn_script_to_json(script)
bn = dag.BayesianNetwork(js)
bn

Name:	Exp1
Nodes:
	n
	b ~ beta(1,1)
	a = 0.5
	c = min([a,b])
	y ~ binom(n,c)

### Single value variable


> VariableName = Number 


In [3]:
SingleValue = bn['a']
print('Node \'a\'')
print(SingleValue)

print('\nFind the value')
print(SingleValue())

Node 'a'
a = 0.5

Find the value
0.5


### Exogenous variable

> VariableName

In [4]:
Exogenous = bn['n']
print('Node \'n\'')
print(Exogenous)

print('\nFind the value; must append external resources')
print(Exogenous({'n': 5}))

Node 'n'
n

Find the value; must append external resources
5


### Random variable

> VariableName ~ p(...)

** p(...) **: a probabilidy density/mass function

In [5]:
Random = bn['b']
print('Node \'b\'')
print(Random)

print('\nSample a value')
print(Random())

Node 'b'
b ~ beta(1,1)

Sample a value
0.451871191663


### Equation output

> OutputName = f(...)

** f(...) **: a mathematical function

In [6]:
Equation = bn['c']
print('Node \'c\'')
print(Equation)

parents = {
    'a': SingleValue(), 
    'b': Random() 
}

print('\nCalculate a value')
print(Equation(parents))

Node 'c'
c = min([a,b])

Calculate a value
0.0835299383188


### Pseudo Node
Coming soon...

## For simulation model

### Basic case

In [51]:
script = '''
PCore Exp2 {
   mu_x = 0
   mu_y = 10
   sd ~ exp(1)
   x ~ norm(mu_x, sd)
   y ~ norm(mu_y, sd)
}
'''

bn = dag.bn_from_script(script)
sc = dag.as_simulation_core(bn)
sg = sc.generate('exp2')
sg

sd: 0.663375, mu_y: 10, mu_x: 0

In [33]:
y = sg.get_sampler('y')

In [49]:
y()

8.808665796452086

### Define exposed nodes

### Heirarchical node groups

### Random effects

### Example 1: beta-binomial model

In [9]:
# Sample 

In [10]:
# Sample with given nodes

### Example 2:  regression model

### Example 3: BMI model

### Example 4: A simple agent-based model 